## **Câu 1 _ Phân bố các loại rừng theo xã ở huyện Di Linh là như thế nào?**

In [133]:
import plotly.express as px

grouped_data = data.groupby(['Commune', 'Land type']).size().reset_index(name='Count')

commune_totals = grouped_data.groupby('Commune')['Count'].sum().reset_index(name='Total')

grouped_data = grouped_data.merge(commune_totals, on='Commune')
grouped_data['Percentage'] = (grouped_data['Count'] / grouped_data['Total']) * 100

grouped_data['Land type adjusted'] = grouped_data.apply(
    lambda row: 'Other' if row['Percentage'] < 10 else row['Land type'], axis=1
)

grouped_data['Commune adjusted'] = grouped_data.apply(
    lambda row: 'Khác' if row['Total'] < 75 else row['Commune'], axis=1
)

final_data = grouped_data.groupby(['Commune adjusted', 'Land type adjusted'])['Count'].sum().reset_index()
final_data.rename(columns={'Commune adjusted': 'Commune'}, inplace=True)

commune_order = final_data.groupby('Commune')['Count'].sum().sort_values(ascending=False).index.tolist()

fig = px.bar(
    final_data,
    y='Commune',
    x='Count',
    color='Land type adjusted',
    title='Phân bố các loại rừng theo xã ở huyện Di Linh (gộp xã nhỏ và loại rừng hiếm)',
    labels={'Commune': 'Xã', 'Count': 'Số lượng lô đất', 'Land type adjusted': 'Loại rừng'},
    color_discrete_sequence=px.colors.qualitative.Pastel,
    orientation='h',
    category_orders={'Commune': commune_order}
)

# Layout
fig.update_layout(
    yaxis_title='Xã',
    xaxis_title='Số lượng lô đất',
    legend_title='Loại rừng',
    showlegend=True,
    hovermode='closest',
    template='plotly_white',
    legend=dict(x=1.02),
    width=1400,
    height=800
)

fig.show()

In [134]:
data['Date'] = pd.to_datetime(data['Date'])
data['YearMonth'] = data['Date'].dt.to_period('M').astype(str)

total_by_type = data['Land type'].value_counts(normalize=True)

rare_threshold = 0.05
rare_types = total_by_type[total_by_type < rare_threshold].index
data['Land type adjusted'] = data['Land type'].apply(lambda x: 'Other' if x in rare_types else x)

grouped_data = data.groupby(['YearMonth', 'Land type adjusted']).size().reset_index(name='Count')

fig = px.area(
    grouped_data,
    x='YearMonth',
    y='Count',
    color='Land type adjusted',
    title='Xu hướng số lượng lô đất theo loại đất qua thời gian (gộp loại hiếm)',
    labels={
        'YearMonth': 'Thời gian (Năm-Tháng)',
        'Count': 'Số lượng lô đất',
        'Land type adjusted': 'Loại đất'
    },
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(
    xaxis_title='Thời gian (Năm-Tháng)',
    yaxis_title='Số lượng lô đất',
    legend_title='Loại đất',
    xaxis_tickangle=45,
    showlegend=True,
    hovermode='x unified',
    template='plotly_white',
    width=1400,
    height=500
)

fig.show()

## **Câu 3 _ Mật độ các lô đất rừng theo khu vực xã như thế nào khi xét theo tọa độ địa lý?**

In [ ]:
# Tạo biểu đồ mật độ trên bản đồ
fig = px.density_mapbox(
    data,
    lat='Latitude',
    lon='Longtitude',
    radius=10,
    center=dict(lat=11.5, lon=108.1),
    zoom=10,
    mapbox_style='open-street-map',
    title='Mật độ các lô đất rừng theo xã',
    hover_data=['Commune', 'Land type'],
    color_continuous_scale=px.colors.sequential.Plasma  # Gradient từ xanh đến đỏ
)

# Tùy chỉnh layout
fig.update_layout(
    margin={'r': 0, 't': 50, 'l': 0, 'b': 0},
    coloraxis_colorbar_title='Mật độ',
    template='plotly_white',
    width=1000,
    height=600
)

# Hiển thị biểu đồ
fig.show()